# 对象池
## 小整数对象池
- 在一个程序中，[-5, 256]之间的数字是经常被使用的，因此python在程序运行过程中，定义好好了一个小整数池，也就是从-5到256之间的数字都是提前在内存中建立好的，不会被系统回收，并且以后就你创建了一个变量是在这个范围之内，那么都是同一个对象.同样对于单个字母也是一样的


In [3]:
a = 1
b = 1
print(id(a), id(b))
print("="*40)

a = 257
B = 257
print(id(a))
print(id(B))

140731697042240 140731697042240
2079450590384
2079450590448


## 对于只含英文字符的字符串，共用同一个对象
- 如果两个字符串的字符都是一样的，并且只包含英文字符，那么会共用同一个对象


In [11]:
a = "hellokity"
b = 'hellokity'
print(id(a), id(b))
import sys
print(sys.getrefcount(a))

c = "I love you so much"
d = 'I love you so much'
print(id(c), id(d))

f = 1
print(sys.getrefcount(f))
# 通过引用计数可以看出，1这个数被python引用了很多次

2079452568048 2079452568048
3
2079451106160 2079452682544
2204


# 析构函数和引用计数
- 1.python中的类也有析构函数，即__del__方法，只要这个对象在内存中即将被消灭的时候，就会调用这个方法
        class Person(object):
            def __del__(self):
                print("我即将被消灭")
        
        p1 = Person()
        # 调用这个方法，就会执行Person中的__del__方法
        del p1
- 2.引用计数
    - python中的对象是使用引用计数的方式实现的，即如果没有任何对象引用到一块内存，那么python就会把这个内存回收
    - sys.getrefcount(a)可以查看a对象的引用计数，但是比正常计数大1，因为调用函数的时候传入a，这会让a的引用计数+1
            class Person():
                def __del__(self):
                    print("我要被消灭了")


            p1 = Person()
            p2 = p1
            # del方法并不会回收这个内存，它只是让引用计数减去1，只有 引用计数为0了，才会真正回收这块内存，即 真正执行__del__方法
            del p1
            # 先会打印====
            print("=====")
            # 执行__del__方法


# Python 垃圾回收机制
### 引用计数
- 在python中，使用了引用计数这一计数实现内存管理，一个对象被创建后就有一个变量指向它，那么就说明它的引用计数为1，以后如果有其他变量指向它，引用计数就会相应的增加，如果将一个变量不再指向这个对象，那么这个对象的引用计数减去1,。如果一个对象没有任何变量指向它，即引用计数为0，那么这个对象就会被python回收

### 循环引用
- 模块之间相互引用有可能会产生循环引用的问题
- 如下不会产生问题：
        a.py文件
        import b
        print("这是a文件")
        
        b.py文件
        import a
        print("这是b文件")
        
        执行b文件得到的结果如下：
        这是b文件
        这是a文件
        这是b文件
        结果解释：执行b文件时，需要先导入a模块，因为'a'不在sys.modules中，所以要先执行a文件的代码，又a文件中需要导入b文件，但在sys.modules中没有'b'，所以要去执行b文件，执行时，需要导入a文件，但之前已经导入过一次了，所以会执行下面的代码（即这是b文件）,然后回到a文件（因为a文件引用了b文件，b文件执行后就会执行a文件），又b文件引用了a文件，当a文件执行完后，又会执行b文件
        执行过程可以概括为：b=>a=>b
- 但是如下就会产生问题：
        a.py文件
        from .b import say_b

        def say_a():
            print("a文件说 ")
        print("这是a文件")
        
        b.py文件
        from .a import say_a

        def say_b():
            print("b文件说")
        print("这是b文件")
        原理和上面的一样，但是上面在导入时没有导入任何函数，但在这里导入了具体的函数。在执行b文件时，发现a没导入，所以导入a，在执行a文件时发现b文件没导入，所以导入b（即执行b），此时a文件已导入，但是里面的函数并没有定义好，所以压根导入不了say_a，所以就会报错
- 解决循环引用的办法：
    - 新建另一个文件 ，将需要引用的那个函数放在那个文件中
- 引用计数虽然可以在一定程度上解决内存管理的问题，但是还是有不能解决的问题，即循环引用，比如现在有两个对象分别为a和b，a指向了b，b又指向了a，那么他们两的引用计数永远都不会为0，所以永远都不会得到回收


### 标记清除和分代回收
- 在python程序中，每次你创建了一个对象，那么就会将这个对象挂到一个叫做零代链表中（当然这个链表是python内部的，python开发者是无法访问的）
- 文章参考 ：https://www.jianshu.com/p/1e375fb40506

# gc模块
- python中的gc模块封装了许多和对象以及垃圾回收相关的方法
### 导致引用计数+1的情况
- 对象被创建，并被一个对象所引用，例如a = 23
- 对象被另外一个变量所引用
- 对象被作为参数传递个函数
- 对象被添加到容器中，比如添加到列表、元组、字典、集合中等


### 导致引用计数-1的情况
- 引用这个对象的变量碑额删除掉了， 例如del a
- 引用这个对象的变量转向其他的对象了
- 函数作用域执行完毕后，比如一个函数中的临时变量，在这个函数执行结束后就会消失
- 对象所在的这个容器被销毁，或者从这个容器中删除了这个对象， 也会导致这个引用计数会减去1

### 查看一个对象的引用计数
- sys.getrefcount

### gc模块常用函数
- 1.gc.get_debug(flagsj):设置gc的debug日志，一般设置为gc.DEBUG_LEAK可以 看到内存泄漏的对象
- 2.gc.collect(generation):手动执行垃圾回收，会将那些循环引用的对象给回收了，这个函数可以传递参数，0代表只回收第0代的垃圾对象，1代表回收第0代和第1代的对象，2代表回收第0,1,2代的对象 ，如果不传递参数，默认使用2
- 3.gc.get_threshold():获取gc模块执行垃圾回收的阈值，返回的是个元组，第0个是零代的阈值，第1个是1代的阈值，第2个是2代的阈值
- 4.gc。set_threshold():设置执行垃圾回收的阈值
- 5.gc.get_count():获取当前自动执行 垃圾回收的计数器，返回一个元组，第0个是零代的垃圾对象的数量，第1个是零代链表遍历的次数，第2个是1代链表遍历的次数

### 关于阈值和垃圾回收
- 假设通过gc.get_threshold()返回的是（700， 10， 10），那么意味着只要零代垃圾值到了700，就会执行gc.collect(0),回收零代的垃圾值，只要1代垃圾值到了10，就会执行gc.collect(1),回收零代和1代的垃圾值，只要2代垃圾值到了10，就会执行gc.collect(2)，回收零代和1代以及2代的垃圾值、

### 注意点
- gc模块不能处理的是，如果两个循环引用的对象都实现了__del__方法，那么将不会进行垃圾回收，因此尽量不要在类中实现自己的__del__方法，否则发生循环引用后就会产生内存泄漏